# I. Tweet Data Cleaning



In [1]:
## Tweet Table

In [2]:
import pymongo
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import json
import time

In [3]:
#function to cheack whether 
def check_var(data, x):
    try:
        data[x]
        return True
    except:
        return False

In [4]:
%%time

#making mongoDB database and collection
client = MongoClient('localhost', 27017)
db = client['mydatabase']

want_drop = input()
if(want_drop == 'yes'): ## if type yes, then it will drop tweet table from mongodb to prepare new importing.
    db.tweet.drop()
    
tweet = db['tweet']

yes
CPU times: user 12.2 ms, sys: 6.68 ms, total: 18.9 ms
Wall time: 25.7 s


In [5]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [6]:
#yelin directory '/Users/yelin/Downloads/corona-out-3/corona-out-3'
#Mo 'C:/Users/maxmo/OneDrive/Desktop/Rutgers/Spring2020/Data Management/corona-out-2'
# Hui Wang  file = '/Users/huiwang/Downloads/corona-out-3'
file = '/Users/huiwang/Downloads/corona-out-3'

In [9]:
%%time

#id, user_id, created_at, text, retweet_count, favorite_count, reply_count, hashtags
import json
with open(file, 'r', encoding = 'utf8') as f1 :
    for line in f1 :
        try :
            data = json.loads(line)
        except :
            continue
        
        hashtag = [] #to collect list of hashtags
        
        if (data['lang'] == 'en'):
            
            #case 1: check whether it is a retweet data or not
            if (check_var(data, 'retweeted_status')) :

                Created_at = pd.to_datetime(data['retweeted_status']['created_at']).strftime('%B %d, %Y, %r')
                UserID = data['retweeted_status']['user']['id']
                ID = data['retweeted_status']['id']
                retweet_count = data['retweeted_status']['retweet_count']
                favorite_count = data['retweeted_status']['favorite_count']
                reply_count = data['retweeted_status']['reply_count']

                #check whether there is extension version of that tweet or not        
                if (check_var(data['retweeted_status'], 'extended_tweet')) :
                    Text = data['retweeted_status']['extended_tweet']['full_text']
                    for t in data['retweeted_status']['extended_tweet']['entities']['hashtags'] :
                        hashtag.append(t['text'])                
                else :
                    Text = data['retweeted_status']['text']
                    for t in data['retweeted_status']['entities']['hashtags'] :
                        hashtag.append(t['text'])

                #check whether it is retweet of quoted tweet or not
                if (check_var(data['retweeted_status'], 'quoted_status')) :

                    #check whether there is extension version of the original tweet or not 
                    if (check_var(data['retweeted_status']['quoted_status'], 'extended_tweet')) :
                        Text = Text + ' || ' + data['retweeted_status']['quoted_status']['extended_tweet']['full_text']
                        for t in data['retweeted_status']['quoted_status']['extended_tweet']['entities']['hashtags'] :
                            hashtag.append(t['text'])                
                    else :
                        Text = Text + ' || ' + data['retweeted_status']['quoted_status']['text']
                        for t in data['retweeted_status']['quoted_status']['entities']['hashtags'] :
                            hashtag.append(t['text'])               

            #case 2: check whether it is a quote tweet data or not
            elif (check_var(data, 'quoted_status')) :
                Created_at = pd.to_datetime(data['created_at']).strftime('%B %d, %Y, %r')
                UserID = data['user']['id']
                ID = data['id']
                retweet_count = data['retweet_count']
                favorite_count = data['favorite_count']
                reply_count = data['reply_count']

                #check whether there is extension version of that quote tweet or not 
                if(check_var(data, 'extended_tweet')) :
                    Text = data['extended_tweet']['full_text'] + ' || ' 
                else:
                    Text = data['text'] + ' || '

                for t in data['entities']['hashtags'] :
                    hashtag.append(t['text'])

                #check whether there is extension version of the originl tweet or not 
                if (check_var(data['quoted_status'], 'extended_tweet')) :
                    Text = Text + data['quoted_status']['extended_tweet']['full_text']
                    for t in data['quoted_status']['extended_tweet']['entities']['hashtags'] :
                        hashtag.append(t['text'])                
                else:
                    Text = Text + data['quoted_status']['text']
                    for t in data['quoted_status']['entities']['hashtags'] :
                        hashtag.append(t['text'])

            #if it does not belong to case 1 and case 2, it is an ordinary tweet data
            else :
                Created_at = pd.to_datetime(data['created_at']).strftime('%B %d, %Y, %r')
                UserID = data['user']['id']
                ID = data['id']
                retweet_count = data['retweet_count']
                favorite_count = data['favorite_count']
                reply_count = data['reply_count']

                #check whether there is extension version of that tweet or not  
                if(check_var(data, 'extended_tweet')) :
                    Text = data['extended_tweet']['full_text']
                else :
                    Text = data['text']

                for t in data['entities']['hashtags'] :
                    hashtag.append(t['text'])

            tw = {'id' : ID,
                  'user_id' : UserID,
                  'created_at' : Created_at,
                  'text' : Text,
                  'retweet_count' : retweet_count,
                  'favorite_count' : favorite_count,
                  'reply_count' : reply_count,
                  'hashtags': hashtag
                 }

            #if the data is already in our database, we update it 
            if (tweet.count_documents({'id' : tw['id']}) > 0) :
                db.tweet.update_one({'id' : tw['id']},
                                    {'$set' : {'retweet_count' : tw['retweet_count'],
                                               'favorite_count' : tw['favorite_count'],
                                               'reply_count' : tw['reply_count']}})
            #if it is not, we insert it    
            else :
                tweet.insert_one(tw)
            

tweet.create_index([ ('id', 1) ])

CPU times: user 1min 37s, sys: 5.64 s, total: 1min 43s
Wall time: 18min 50s


'id_1'

In [10]:
tweet_table = pd.DataFrame(tweet.find())
tweet_table.shape

(19076, 9)

In [11]:
tweet_table.head(5)

,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags
0,5ea7785aa8a68922687753f6,1253949413191344128,207809313,"April 25, 2020, 07:30:12 AM",India’s war with Corona is ongoing.\n\nPlay yo...,398,2144,108,[FeedTheNeedy]
1,5ea7785aa8a68922687753f7,1253992905703862272,858859031464751104,"April 25, 2020, 10:23:01 AM",VIDEO | 25.04.20\n\n&gt; Monsta X appears on Y...,59,144,1,"[몬스타엑스, MONSTAX, 판타지아엑스, FANTASIA_X]"
2,5ea7785aa8a68922687753f8,1253697983343816706,2719753171,"April 24, 2020, 02:51:06 PM","Sri ByReddy Rajasekhar Reddy garu,a leader,who...",2238,7243,47,[]
3,5ea7785aa8a68922687753f9,1254022773598572544,1091660129894838272,"April 25, 2020, 12:21:42 PM",@VinceMcMahon @TripleH We hereby honor to anno...,0,0,0,[]
4,5ea7785aa8a68922687753fa,1254022776094105602,375777294,"April 25, 2020, 12:21:43 PM",im making 17 a hr doing nothing 😭😭😭😭 i love th...,0,0,0,[]






# II. User Data Cleaning





In [12]:
## User Table

In [13]:
import pandas as pd
import psycopg2
import json

In [14]:
##Creating the database
conn = psycopg2.connect("dbname=postgres port=5432 user=postgres password=password")
conn.set_session(autocommit = True)
cur = conn.cursor()

cur.execute("CREATE DATABASE twitter_users" )
conn.commit()
cur.close()
conn.close()

DuplicateDatabase: database "twitter_users" already exists


In [15]:
#Creating the table
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()

cur.execute("CREATE TABLE IF NOT EXISTS users_data( user_id bigint PRIMARY KEY, name varchar(255), user_name varchar(255), verified_status boolean,followers_count integer,friends_count integer,statuses_count integer,user_location varchar(500),favourites_count integer);" )
conn.commit()
cur.close()
conn.close()

In [16]:
def check_var(data,x):
    try:
        data[x]
        return True
    except:
        return False

In [17]:
%%time

#This function takes the the variable inputs and stores it into a PostgreSQL database
def store_data(user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count):
    conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
    cur = conn.cursor()
    insert_query = "INSERT INTO users_data (user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (user_id) DO UPDATE SET verified_status=EXCLUDED.verified_status, followers_count=EXCLUDED.followers_count, friends_count=EXCLUDED.friends_count, statuses_count=EXCLUDED.statuses_count, user_location=EXCLUDED.user_location, favourites_count=EXCLUDED.favourites_count"
    cur.execute(insert_query, (user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location,favourites_count))
    conn.commit()
    cur.close()
    conn.close()
    
    
# Decoding the JSON file
with open(file, 'r', encoding = 'utf8') as f1:
    for line in f1:
        
        try:
            data = json.loads(line)
        except:
            continue
        
        if (data['lang'] == 'en'): # only 
            
            if (check_var(data, 'retweeted_status')):

                user_id = data['retweeted_status']['user']['id']
                name = data['retweeted_status']['user']['name']
                user_name = data['retweeted_status']['user']['screen_name']
                verified_status = data['retweeted_status']['user']['verified']
                followers_count = data['retweeted_status']['user']['followers_count']
                friends_count = data['retweeted_status']['user']['friends_count']
                statuses_count = data['retweeted_status']['user']['statuses_count']
                user_location = data['retweeted_status']['user']['location']
                favourites_count= data['retweeted_status']['user']['favourites_count']


            else :
                user_id = data['user']['id']
                name = data['user']['name']
                user_name = data['user']['screen_name']
                verified_status = data['user']['verified']
                followers_count = data['user']['followers_count']
                friends_count = data['user']['friends_count']
                statuses_count = data['user']['statuses_count']
                user_location = data['user']['location']
                favourites_count= data['user']['favourites_count']


            #insert the data into the PostgreSQL database
            store_data(user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count)

CPU times: user 41.1 s, sys: 17.2 s, total: 58.3 s
Wall time: 8min 33s


In [18]:
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()


# A function that takes in a PostgreSQL query and outputs a pandas database 
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [42]:
user_table = create_pandas_table("SELECT * FROM users_data")
tweet_table.shape

(19076, 9)

In [43]:
user_table.head(5)

,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,606475390,king-Aleks👑🔬💉,emmanuel_ceejay,False,7673,7025,15375,"Abuja, Nigeria",38228
1,865733987561381888,ANH,BritishKatieKim,False,143,882,19131,None,89339
2,477012156,Liddle’ Savage,littledeekay,False,57051,1226,37067,None,123993
3,1132273796138905600,Terri Kamp,RampTheresa,False,1522,256,20861,None,29166
4,130684893,Andy McCarthy,AndrewCMcCarthy,True,246900,1204,13366,"New York, NY",47






# III. Searching Application





Implement 1

In [52]:
%%time
# using MongoDB
# most popular original tweet ranked by the retweet_count, reply_count, and favorite_count 
pop_tweet = pd.DataFrame(tweet.find({},{'user_id':1,'created_at':1,'text':1,'retweet_count':1,'favorite_count':1,'reply_count':1,'hashtags':1}).sort([("retweet_count", -1),("reply_count", -1),("favorite_count", -1)]))
pop_tweet

CPU times: user 297 ms, sys: 17.2 ms, total: 314 ms
Wall time: 741 ms


,_id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags
0,5ea779b8a8a689226877756a,1100261477989126145,"March 13, 2020, 12:43:40 AM",*corona virus enters my body*\n\nThe 4 Flintst...,237307,811062,1811,[]
1,5ea7792ba8a6892268776eb9,1112592502727548928,"March 18, 2020, 05:51:18 PM",When this Corona shit passes we have to promis...,181584,764405,1595,[]
2,5ea7789fa8a6892268776337,1131227186,"March 10, 2020, 05:52:14 PM",THIS MAN IS A GENIUS he figured out the Corona...,179037,515867,1578,[]
3,5ea77a49a8a6892268777b08,728371324184301568,"March 16, 2020, 11:04:57 PM",“corona time “😭😭😭😭 https://t.co/iXBMHVcFoY,132003,422685,1516,[]
4,5ea77c69a8a68922687799e7,716432228,"March 23, 2020, 05:13:35 PM",This is Dr. Usama Riaz. He spent past weeks sc...,119601,331618,4876,[]
...,...,...,...,...,...,...,...,...
19071,5ea77cc4a8a6892268779e75,2491804962,"April 25, 2020, 02:48:36 PM",40 Coronavirus Cases In Milwaukee County Linke...,0,0,0,[]
19072,5ea77cc4a8a6892268779e76,892116055048548354,"April 25, 2020, 02:48:36 PM",The good news! He can now vote 100 times... ||...,0,0,0,[]
19073,5ea77cc4a8a6892268779e77,734263483173212160,"April 25, 2020, 02:48:37 PM",i will be makin tacos and gettin drunk on cinc...,0,0,0,[]
19074,5ea77cc4a8a6892268779e78,3130596629,"April 25, 2020, 02:48:37 PM",We have 1/4 of all deaths from #Corona - thank...,0,0,0,"[Corona, ashamed]"


In [53]:
# TOP 10 most popular hashtag ranked by aggregating the lists of hashtags (case insensitive)
from bson.son import SON
pipeline = [
     {"$unwind": "$hashtags"},
     {"$group": {"_id": {"$toLower": "$hashtags"}, "count": {"$sum": 1}}},
     {"$sort": SON([("count", -1), ("_id", -1)])}
] 

import pprint
pprint.pprint(list(db.tweet.aggregate(pipeline))[0:10])

[{'_id': 'corona', 'count': 1248},
 {'_id': 'covid19', 'count': 542},
 {'_id': 'coronavirus', 'count': 384},
 {'_id': 'covid_19', 'count': 292},
 {'_id': 'lockdown', 'count': 166},
 {'_id': 'covid', 'count': 129},
 {'_id': 'indiafightscorona', 'count': 119},
 {'_id': 'संतरामपालजी_का_सत्संग_सुनें', 'count': 114},
 {'_id': 'stayhome', 'count': 102},
 {'_id': 'covidー19', 'count': 72}]


Implement 2

In [54]:
# using SQL
# the most active verified user ranked by statuses_count
active_user = create_pandas_table("SELECT * FROM users_data WHERE verified_status = 'True' ORDER BY statuses_count DESC")
active_user.head(5)

,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,47596019,Liputan6.com,liputan6dotcom,True,3687560,693,1381273,Jakarta Indonesia,7062
1,16973333,The Independent,Independent,True,3202840,1146,976895,"London, England",3
2,378809160,𝔾𝕀𝔻𝕀𝕋ℝ𝔸𝔽𝔽𝕀ℂ,Gidi_Traffic,True,1676895,5826,922265,Everywhere,14478
3,2475273985,Paytm Care,Paytmcare,True,248675,15,861933,India,402
4,15448383,Inquirer,inquirerdotnet,True,3076056,1852,858348,Makati City,34260


In [55]:
# the most popular verified user ranked by followers_count
pop_user = create_pandas_table("SELECT * FROM users_data WHERE verified_status = 'True' ORDER BY followers_count DESC")
pop_user.head(5)

,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,18839785,Narendra Modi,narendramodi,True,55786179,2364,26559,India,0
1,807095,The New York Times,nytimes,True,46361159,904,393823,New York City,18483
2,145125358,Amitabh Bachchan,SrBachchan,True,41596464,1833,66428,"Mumbai, India",75
3,101311381,Shah Rukh Khan,iamsrk,True,40026738,77,58040,None,32
4,471741741,PMO India,PMOIndia,True,34461808,486,25996,India,0


In [56]:
# the most active location ranked by tweets count
pop_location = create_pandas_table("SELECT user_location, count(*) as amount FROM users_data WHERE user_location != 'None' GROUP BY user_location ORDER BY amount DESC")
pop_location.head(5)

,user_location,amount
0,India,363
1,"New Delhi, India",185
2,United States,125
3,"Mumbai, India",121
4,"Lagos, Nigeria",112






Join the tweet table and user table

In [57]:
%%time
# using Pandas
full_table = pd.merge(tweet_table, user_table, how = 'left', on = ['user_id'])
full_table

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

Implement 3

In [58]:
%%time

# using Pandas
# search the tweets that contain the keyword 'election'
election = full_table[full_table['text'].str.contains('election',case=False)].drop(['_id','id','user_id'], axis=1)

# rank the tweets that contain the keyword 'election' by favorite_count, retweet_count and reply_count
election.sort_values(by = ['favorite_count', 'retweet_count', 'reply_count'], ascending = (False, False, False))

NameError: name 'full_table' is not defined

In [ ]:
%%time
## user interactive search for text (case insensitive)
search_keyword = input()
user_search = full_table[full_table['text'].str.contains(search_keyword,case=False)].drop(['_id','id','user_id'], axis=1)
user_search.sort_values(by = ['favorite_count', 'retweet_count', 'reply_count'], ascending = (False, False, False))

Implement 4

In [ ]:
%%time
# using Pandas
# search for a hashtag 'lockdown' (case insensitive)
target = ['lockdown']
lockdown = full_table[pd.DataFrame(full_table.hashtags.tolist()).applymap(lambda s:s.lower() if type(s) == str else s).isin(target).any(1)].drop(['_id','id','user_id'], axis=1)

# rank the tweets that contain the hashtag 'lockdown' by followers_count
lockdown.sort_values(by = ['followers_count'], ascending = (False))

In [89]:
%%time
## user interactive search for hashtag (case insensitive)

search_keyword = input()

target = [search_keyword]
lockdown = full_table[pd.DataFrame(full_table.hashtags.tolist()).applymap(lambda s:s.lower() if type(s) == str else s).isin(target).any(1)].drop(['_id','id','user_id'], axis=1)
lockdown.sort_values(by = ['followers_count'], ascending = (False))

quarantine
Wall time: 1.25 s


,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
351,"April 25, 2020, 11:39:51 AM",Chalo use this sound and tag me #sanitizer #corona #lockdown #SocialDistancing #quarantine https://t.co/2Hiir1U2VC,12,171,11,"[sanitizer, corona, lockdown, SocialDistancing, quarantine]",Shefali Bagga,shefali_bagga,False,87572,10,7243,"Delhi, India",3624
5369,"April 25, 2020, 12:59:43 PM","My dear friend and bandmate Giulio is doing an unplugged livestream of originals and covers tonight at 6pm UK Time/7pm CET over on his other band’s (@Drones72band) Facebook page.\n\nIt’ll be excellent, and it’s his 1st post-Corona performance, so please tune in if you can. || Voice and breathe are back. So I’ll do home unplugged tonight. Streaming live at 7pm CET from band’s page:\nhttps://t.co/9BkH6Jqe7z #homeconcert #live #quarantine #aloneathome #QuarantineLife #isolation #lockdown #unplugged #acoustic #livestream #Covid_19 #StayHome #StaySafe https://t.co/1bHyqDwTar",2,8,1,"[homeconcert, live, quarantine, aloneathome, QuarantineLife, isolation, lockdown, unplugged, acoustic, livestream, Covid_19, StayHome, StaySafe]",Steve Bullock,GuitarMoog,False,62263,8273,75366,"Brussels, Belgium",123095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16189,"April 25, 2020, 02:26:25 PM",This seems #sensible\n#sarcasm #sarcasticmemes #economy #indian #corona #quarantine #funny https://t.co/XizLOsMMTG,0,0,0,"[sensible, sarcasm, sarcasticmemes, economy, indian, corona, quarantine, funny]",volatile_memers,MemersVolatile,False,1,1,11,None,2
1327,"April 25, 2020, 12:29:23 PM",Quarantine Life!☀️🍑💧 #COVID2019 #level4lockdown #Covid_19 #day30oflockdown #Corona #Quarantine #coronavirus #Twitter #follow https://t.co/EJNsUMB9bK,0,0,0,"[COVID2019, level4lockdown, Covid_19, day30oflockdown, Corona, Quarantine, coronavirus]",Sonia,TweetsSonya,False,0,0,6,"Los Angeles, CA",0


Implement 5

In [ ]:
# Time
# We got the tweet data and corresponding user data in one time slot, but we save the data based on the original tweet
# try to search the earliest created original tweet that is still getting retweeted in the time slot.

In [90]:
%%time
full_table['created_at'] = pd.to_datetime(full_table['created_at'])
full_table.sort_values(by = ['created_at'], ascending = (True))

,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
14241,5ea4917b7fc2050f80995f6c,341578219176214528,1242575784,2013-06-03 15:32:48,Corona virus....its coming,73776,131225,9968,[],Marco,Marco_Acortes,False,5652,106,3839,None,4842
6794,5ea4903b7fc2050f80994255,872849401928179713,189011442,2017-06-08 16:14:48,"But to return to a moment of grace.\n\n""Take me to the Seine, let us gaze into it until we become little fishes &amp; recognize each other again."" https://t.co/WfLNI92buL || ""I have often reflected that 'Corona' is your most beautiful poem""\nIB, 6 June 1949\n\nI think so, too-with Celan echoing Rilke, 'es ist Zeit.' https://t.co/CS41BlJ4jw",16,39,0,[],coldplums,coldplums,False,4341,1435,24661,on the road,41167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19073,5ea492b47fc2050f8099724c,1254059746652389378,734263483173212160,2020-04-25 14:48:37,i will be makin tacos and gettin drunk on cinco de mayo by myself if i have to fuck a bitch named corona,0,0,0,[],dirty d 😽💙,deannarae21,False,684,482,43705,baerobbie 💙,75847
19075,5ea492b57fc2050f8099724e,1254059751379361793,2279976427,2020-04-25 14:48:38,@VSReddy_MP I think wantedly you ate damaging your leaders image @ysjagan One side there is an increase of Corona cases and also @srikakulamdistrict now attacked with corona on the other side you are giving these tweets How to understand it @ArogyaAndhra @AndhraPradeshCM @apfightscorona,0,0,0,[],Venugopi,Venu7630,False,3,35,42,None,83
